In [5]:
import pandas as pd

df = pd.read_excel(r"C:\Users\POTENCIA\OneDrive - POTENCIA\Documents\TAREA_ENTIDADES\data\00_raw\secop\secop_obras_2019_2025.xlsx")

In [ ]:
def resumen_unicos(df):
    """
    Imprime los valores únicos y su conteo para cada columna del DataFrame.
    """
    for col in df.columns:
        print(f"\n📌 Columna: {col}")
        print("-" * (10 + len(col)))
        
        # Conteo de valores únicos
        conteo = df[col].value_counts(dropna=False)
        
        # Imprimir resultados
        for valor, cantidad in conteo.items():
            print(f"{valor}: {cantidad}")


In [8]:
df.columns

Index(['nombre_entidad', 'nit_entidad', 'codigo_entidad', 'orden', 'sector',
       'id_contrato', 'estado_contrato', 'tipo_de_contrato',
       'modalidad_de_contratacion', 'codigo_de_categoria_principal',
       'descripcion_del_proceso', 'fecha_de_inicio_del_contrato',
       'fecha_de_fin_del_contrato', 'tipodocproveedor', 'documento_proveedor',
       'codigo_proveedor', 'proveedor_adjudicado', 'origen_de_los_recursos',
       'valor_del_contrato', 'c_digo_bpin', 'urlproceso',
       'presupuesto_general_de_la_nacion_pgn',
       'sistema_general_de_participaciones', 'sistema_general_de_regal_as',
       'recursos_propios_alcald_as_gobernaciones_y_resguardos_ind_genas_',
       'recursos_de_credito', 'recursos_propios', 'objeto_del_contrato'],
      dtype='object')

In [11]:
a = df[['nombre_entidad', 'sector',
        'estado_contrato',
        'codigo_de_categoria_principal',
      'proveedor_adjudicado',  'c_digo_bpin']]
resumen_unicos(a)


📌 Columna: nombre_entidad
------------------------
INVIAS: 2685
AEROCIVIL: 761
DIRECCION DE IMPUESTOS Y ADUANAS NACIONALES*: 181
CENTRAL ADMINISTRATIVA ESPECIALIZADA INGENIEROS CENAC INGENIEROS: 122
FONDO ROTATORIO DE LA REGISTRADURIA: 111
SUPERINTENDENCIA DE NOTARIADO Y REGISTRO: 90
UNIDAD DE SERVICIOS PENITENCIARIOS Y CARCELARIOS - USPEC: 89
CORPORACION NASA KIWE: 84
FUERZA AEROESPACIAL COLOMBIANA: 76
Ministerio de las Culturas, las Artes y los Saberes: 74
EDUNA (COMPRADOR): 74
ALCALDÍA MUNICIPAL DE ROLDANILLO: 53
INSTITUTO COLOMBIANO AGROPECUARIO - ICA: 53
ALCALDÍA MUNICIPAL DE MAGANGUE: 51
INSTITUTO NACIONAL DE MEDICINA LEGAL Y CIENCIAS FORENSES: 51
ENTerritorio S.A: 48
MINISTERIO DEL DEPORTE: 46
INSTITUTO DE CASAS FISCALES DE EJERCITO: 45
AGENCIA LOGISTICA DE LAS FUERZAS MILITARES: 44
CENAC TOLEMAIDA: 42
MUNICIPIO DE ENVIGADO*: 39
SENA TOLIMA 1: 38
CENAC AVIACION: 37
ARMADA NACIONAL - BASE NAVAL No.1 ARC BOLÍVAR: 37
CVC: 35
SENA REGIONAL SANTANDER Grupo de Apoyo Administrativo Mi